# Spam and ham NLP using Spark

In [1]:
import findspark
findspark.init("/home/ratzz/spark-2.4.4-bin-hadoop2.7")
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [2]:
data = spark.read.csv("smsspamcollection/SMSSpamCollection",inferSchema=True,sep="\t")

In [6]:
data = data.withColumnRenamed("_c0","class").withColumnRenamed("_c1","text") #changing name of column

In [5]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [7]:
from pyspark.sql.functions import length

In [8]:
data = data.withColumn("length",length(data["text"]))

In [9]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [10]:
data.groupBy("class").mean().show() #diffrences in length of ham and spam 

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [12]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover,CountVectorizer,IDF,StringIndexer,VectorAssembler

In [13]:
tokenizer = Tokenizer(inputCol="text",outputCol="token_text")
stop_remove = StopWordsRemover(inputCol="token_text",outputCol="stop_token")
count_vec = CountVectorizer(inputCol="stop_token",outputCol="c_vec")
idf = IDF(inputCol="c_vec",outputCol="tf_idf")
ham_spam_to_numeric = StringIndexer(inputCol="class",outputCol="label")
clean_up = VectorAssembler(inputCols=["tf_idf","length"],outputCol="features")

In [14]:
from pyspark.ml.classification import NaiveBayes

In [15]:
nb = NaiveBayes()

In [16]:
from pyspark.ml import Pipeline

In [17]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_numeric,tokenizer,stop_remove,count_vec,idf,clean_up])

In [18]:
cleaner = data_prep_pipe.fit(data)

In [19]:
clean_data = cleaner.transform(data)

In [20]:
clean_data.columns

['class',
 'text',
 'length',
 'label',
 'token_text',
 'stop_token',
 'c_vec',
 'tf_idf',
 'features']

In [21]:
final_data = clean_data.select("label","features")

In [22]:
final_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [23]:
train ,test = final_data.randomSplit([0.7,0.3])

In [24]:
spam_detector = nb.fit(train)

In [25]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [26]:
test_results = spam_detector.transform(test)

In [27]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,7,8...|[-794.30166698109...|[1.0,7.6879935527...|       0.0|
|  0.0|(13424,[0,1,2,41,...|[-1083.0809984413...|[1.0,1.1443634326...|       0.0|
|  0.0|(13424,[0,1,7,8,1...|[-881.54102118044...|[1.0,1.5400628855...|       0.0|
|  0.0|(13424,[0,1,7,8,1...|[-1202.7659129362...|[1.0,1.5075501846...|       0.0|
|  0.0|(13424,[0,1,12,33...|[-442.35303312821...|[1.0,6.7661961903...|       0.0|
|  0.0|(13424,[0,1,14,18...|[-1363.3270077999...|[1.0,1.3759829062...|       0.0|
|  0.0|(13424,[0,1,14,31...|[-216.97619385832...|[1.0,3.2040759970...|       0.0|
|  0.0|(13424,[0,1,18,20...|[-833.24526307799...|[1.0,6.3480164570...|       0.0|
|  0.0|(13424,[0,1,21,27...|[-751.90752720858...|[1.0,8.7811839619...|       0.0|
|  0.0|(13424,[0

In [28]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [29]:
acc_eval = MulticlassClassificationEvaluator()

In [30]:
acc = acc_eval.evaluate(test_results)

In [31]:
print("acc of nb model")
print(acc)

acc of nb model
0.9228987632962887


# Made By Ratnakar Maurya